In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/adjective-trainer/Adjectives_6of8/Adjectives/87. hot/MVI_5295.MOV
/kaggle/input/adjective-trainer/Adjectives_6of8/Adjectives/87. hot/MVI_5138.MOV
/kaggle/input/adjective-trainer/Adjectives_6of8/Adjectives/87. hot/MVI_5297.MOV
/kaggle/input/adjective-trainer/Adjectives_6of8/Adjectives/87. hot/MVI_5136.MOV
/kaggle/input/adjective-trainer/Adjectives_6of8/Adjectives/87. hot/MVI_9408.MOV
/kaggle/input/adjective-trainer/Adjectives_6of8/Adjectives/87. hot/MVI_9329.MOV
/kaggle/input/adjective-trainer/Adjectives_6of8/Adjectives/87. hot/MVI_5218.MOV
/kaggle/input/adjective-trainer/Adjectives_6of8/Adjectives/87. hot/MVI_9247.MOV
/kaggle/input/adjective-trainer/Adjectives_6of8/Adjectives/87. hot/MVI_9249.MOV
/kaggle/input/adjective-trainer/Adjectives_6of8/Adjectives/87. hot/MVI_9248.MOV
/kaggle/input/adjective-trainer/Adjectives_6of8/Adjectives/87. hot/MVI_5137.MOV
/kaggle/input/adjective-trainer/Adjectives_6of8/Adjectives/87. hot/MVI_9488.MOV
/kaggle/input/adjective-trainer/Adjectiv

In [23]:
import os
import regex as re
import pandas as pd
path='/kaggle/input/adjective-trainer'
lf=[]
df = pd.DataFrame(columns=['File','desc'])
for (root,dirs,files) in os.walk(path,topdown=True):
    
    l=[([i for i in files],root)]
    lf.append(l)

In [24]:
l=[]
for i in range(len(lf)) :
    
     for j in lf[i][0][0]:
         l.append([j,lf[i][0][1]])

In [25]:
df=pd.DataFrame(l,columns=['file','root'])
df.shape

(791, 2)

In [26]:
df['path']=df['root']+'/'+df['file']

In [27]:
l=[]
for i in df.root:
    x=i.split('/')[-1]
    x=x.split('.')[-1]
    if x=='Extra' or x=='extra':
         x=i.split('/')[-2]
         x=x.split('.')[-1]
        
    l.append(x)

In [28]:
df.insert(2, "Label", l, True)

In [29]:
df.head()

,file,root,Label,path
0,MVI_5295.MOV,/kaggle/input/adjective-trainer/Adjectives_6of...,hot,/kaggle/input/adjective-trainer/Adjectives_6of...
1,MVI_5138.MOV,/kaggle/input/adjective-trainer/Adjectives_6of...,hot,/kaggle/input/adjective-trainer/Adjectives_6of...
2,MVI_5297.MOV,/kaggle/input/adjective-trainer/Adjectives_6of...,hot,/kaggle/input/adjective-trainer/Adjectives_6of...
3,MVI_5136.MOV,/kaggle/input/adjective-trainer/Adjectives_6of...,hot,/kaggle/input/adjective-trainer/Adjectives_6of...
4,MVI_9408.MOV,/kaggle/input/adjective-trainer/Adjectives_6of...,hot,/kaggle/input/adjective-trainer/Adjectives_6of...


In [30]:
dfn=pd.DataFrame(df[['path','Label']])

In [31]:
dfn
dfn.to_csv('label.csv', index=False)

In [32]:
!pip install mediapipe
!pip install opencv-python

In [33]:
import mediapipe as mp
import cv2

In [34]:

body = {
    "nose": 0,
    "neck": -1,
    "rightey": 5,
    "leftey": 2,
    "rightea": 8,
    "leftea": 7,
    "rightsho": 12,
    "leftsho": 11,
    "rightel": 14,
    "leftel": 13,
    "rightw": 16,
    "leftw": 15
}
#el->elbow,tw->
hand = {
    "wirst" : 0,
    "index1": 8,
    "index2": 7,
    "index3": 6,
    "index4": 5,
    "middle1": 12,
    "middle2": 11,
    "middle3": 10,
    "middle4": 9,
    "ring1": 16,
    "ring2": 15,
    "ring3": 14,
    "ring4": 13,
    "little1": 20,
    "little2": 19,
    "little3": 18,
    "little4": 17,
    "thumb1": 4,
    "thumb2": 3,
    "thumb3": 2,
    "thumb4": 1
}

**There is a inference that if left shoulder and right shoulder are visible along with nose then we can conclude that the neck is also visible **

# creating a dataframe 


In [35]:

column_names = []
#column_names.append('video_id')
for id_name in body.keys():
    for xy in ["_X", "_Y"]:
        column_names.append(id_name + xy)

for lr in ["_Right", "_Left"]:
    for id_name in hand.keys():
        for xy in ["_X", "_Y"]:
            column_names.append(id_name + lr + xy)

#column_names.append('labels')
posedatadf = pd.DataFrame(columns=column_names)
#creating a dataframe to append frames result 


In [36]:
def multiple_detection(image,model):
    
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

In [37]:
def poseneck(pose_results):
            ls = pose_results.pose_landmarks.landmark[11]
            rs = pose_results.pose_landmarks.landmark[12]
            no = pose_results.pose_landmarks.landmark[0]
            if (ls.visibility > 0.5) & (rs.visibility > 0.5) & (no.visibility > 0.5):
                # This indicates the neck better. But it does not affect the result.
                cx = (ls.x + rs.x) / 2
                cy = (ls.y + rs.y) / 2
                dx = no.x - cx
                dy = no.y - cy
                x = cx + 0.3 * dx
                y = cy + 0.3 * dy
                # x = (ls.x+rs.x)/2
                # y = (ls.y+rs.y)/2
            else:
                x = 0
                y = 0
            return [x, y]

In [38]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils #Draw utilities 

** the idea is to map dictionary to pose dataframe so that features could be converted to tensor accordingly**

In [39]:
def extract_keypoints(results,l:dict):
   
    lt='_X'
    rt='_Y'
    for name,idx in body.items():#extracting body features 
        if results.pose_landmarks:
            if idx==-1:
                res=poseneck(results)
                l[name+lt].append(res[0])
                l[name+rt].append(res[1])
                #body_coor.append(neck)
            else:
                 res=results.pose_landmarks.landmark[idx] 
                 #body_coor.append([res.x,res.y])
                 l[name+lt].append(res.x)
                 l[name+rt].append(res.y)
                
        else:
            l[name+lt].append(0)
            l[name+rt].append(0)
              
    for name,idx in hand.items():#extracting lefthand  features
        lef='_Left'
        if results.left_hand_landmarks:
              res=results.left_hand_landmarks.landmark[idx]
              l[name+lef+lt].append(res.x)
              l[name+lef+rt].append(res.y)
              
        else :
                 l[name+lef+lt].append(0)
                 l[name+lef+rt].append(0)
             
        
              #left_coor.append([res.x,res.y])    
    
    
    for name,idx in hand.items():#extracting righthand  features 
       rig='_Right'
       if  results.right_hand_landmarks:
            res=results.right_hand_landmarks.landmark[idx]
            l[name+rig+lt].append(res.x)
            l[name+rig+rt].append(res.y)
           
       else :
                l[name+rig+lt].append(0)
                l[name+rig+rt].append(0)
                #right_coor.append([res.x,res.y])    
    
    
    

In [40]:
import matplotlib.pyplot as plt

In [41]:
def draw_landmarks(image, results,original_image):
    mp_drawing.draw_landmarks(
      image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(
      image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(
      image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(
      image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)# Draw right hand connections
    plt.figure(figsize=[22,22])
    plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Input Image");plt.axis('off');
    plt.subplot(122);plt.imshow(original_image[:,:,::-1]);plt.title("Pose detected Image");plt.axis('off');
      
#use this function to change colour of your function 

"""def draw_styled_landmarks(image, results,original_image):
    # Draw face connections
    mp_drawing.draw_landmarks(
      image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
      mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
      mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )
    plt.figure(figsize=[22,22])
    plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Input Image");plt.axis('off');
    plt.subplot(122);plt.imshow(original_image[:,:,::-1]);plt.title("Pose detected Image");plt.axis('off');"""

def detectpose( path):
    cap=cv2.VideoCapture(path)
    with mp_holistic.Holistic(min_detection_confidence=0.6,min_tracking_confidence=0.6) as holistic:
         while cap.isOpened():
                
             ret, image = cap.read()
             
             cap.set(1,11)
             ret, image = cap.read()
             image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
             image.flags.writeable = False
             image,results = multiple_detection(image,holistic)
             original_image = image.copy()
             draw_landmarks(image, results,original_image)
             break     #only to justify pose detection                                                                        
    
    

In [42]:
import numpy as np
#creating a empty dictionary for storing sequrnces of frames 

    
from tqdm import tqdm

In [43]:
for path in  tqdm(dfn.path):    


    
    
   
    
  count=0
  cap=cv2.VideoCapture(path)
  new={key: [] for key in column_names}
  with mp_holistic.Holistic(min_detection_confidence=0.6,min_tracking_confidence=0.6) as holistic:
       while cap.isOpened():
            ret, image = cap.read()
            image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) #Color convertion 
            image.flags.writeable = False
            image,results = multiple_detection(image,holistic)
            video_currfrm = cap.get(cv2.CAP_PROP_POS_FRAMES)
            end = cap.get(cv2.CAP_PROP_FRAME_COUNT)   
           
        
            extract_keypoints(results,new)
            count+=1
            if video_currfrm == end:
                 while count <60:
                      extract_keypoints(results,new)
                      count+=1
                 #new =normalizebody(new)
                 #new=normalizehands(new)
                  
                 break
       posedatadf=posedatadf.append(new,ignore_index=True) 
      #print(type(re))
  cap.release()
print(posedatadf.shape)
posedatadf['labels']=df['Label']
posedatadf['video_id']=df['file']
posedatadf.to_csv('indiansign.csv', index=False)
  



100%|██████████| 791/791 [1:02:36<00:00,  4.75s/it]


(791, 108)


In [49]:
type(posedatadf.iloc[1,1][0])

float

In [ ]:
posedatadf.head()

In [ ]:
detectpose(dfn.path[0])


In [ ]:
detectpose(dfn.path[17])

In [ ]:
detectpose(dfn.path[96])

In [ ]:
dfn.to_csv('forlabel.csv', index=False)

In [ ]:
type(posedatadf['nose_X'][0])

In [ ]:
def converter (s:list):
  s,pop(0)
  x=[float(i) for i in s]  
  x=np.asarray(x)
  return x

In [ ]:
label=posedatadf['labels']
ids=posedatadf['video_id']
posedatadf=posedatadf.drop(columns=['labels','video_id'])


In [ ]:
posedatadf = posedatadf.applymap(converter)

In [ ]:
cap=cv2.VideoCapture(dfn.path[0])
count=0
new={key: [] for key in column_names}
with mp_holistic.Holistic(min_detection_confidence=0.6,min_tracking_confidence=0.6) as holistic:
       while cap.isOpened():
            ret, image = cap.read()
            image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) #Color convertion 
            image.flags.writeable = False
            image,results = multiple_detection(image,holistic)
            video_currfrm = cap.get(cv2.CAP_PROP_POS_FRAMES)
            end = cap.get(cv2.CAP_PROP_FRAME_COUNT)   
           
        
            extract_keypoints(results,new)
            count+=1
            if video_currfrm == end:
                 while count <60:
                      extract_keypoints(results,new)
                      count+=1
                 #new =normalizebody(new)
                 #new=normalizehands(new)
                  
                 break
       posedatadf=posedatadf.append(new,ignore_index=True) 
      #print(type(re))
cap.release()
print(posedatadf.shape)
  

In [ ]:
posedatadf